In [1]:
import sqlite3
import pandas as pd
import json
from progressbar import ProgressBar

# Read Lablels

In [3]:
path = '../data/NELA2019/data/labels.csv'
labels_data = pd.read_csv(path)
labels_data.dropna(subset = ["aggregated_label"], inplace=True)

In [4]:
labels_data.head()

,source,aggregated_label,"Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake",...,"Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stcenturywire,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,abcnews,0.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,...,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,activistpost,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,addictinginfo,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
5,ageofautism,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
with open('../data/NELA2019/data/source-metadata.json') as file:
    metadata = (json.loads(file.read()))

In [6]:
for source in metadata.keys():
    labels_data.loc[labels_data.source == source, 'n_articles'] = metadata[source]['articles']

In [7]:
cols = ['source', 'n_articles', 'aggregated_label', 'Media Bias / Fact Check, label','Allsides, bias_rating','BuzzFeed, leaning',]
labels_data[cols].head(10)

,source,n_articles,aggregated_label,"Media Bias / Fact Check, label","Allsides, bias_rating","BuzzFeed, leaning"
0,21stcenturywire,791.0,2.0,conspiracy_pseudoscience,NaN,left
1,abcnews,2861.0,0.0,left_center_bias,Lean Left,NaN
2,activistpost,2780.0,2.0,conspiracy_pseudoscience,NaN,left
3,addictinginfo,12.0,1.0,left_bias,NaN,left
5,ageofautism,646.0,2.0,conspiracy_pseudoscience,NaN,NaN
6,airwars,16.0,0.0,least_biased,NaN,NaN
7,aljazeera,11611.0,0.0,left_center_bias,Center,NaN
8,americablognews,104.0,0.0,left_bias,NaN,left
14,bbcuk,12211.0,0.0,left_center_bias,NaN,NaN
15,bearingarms,1902.0,0.0,right_bias,NaN,right


# Read Data

In [8]:
# Execute query and load results into pandas dataframe
def execute_query_pandas(path, query):
    conn = sqlite3.connect(path)
    df = pd.read_sql_query(query, conn)
    return df

In [9]:
def load_articles(path, labels_data, alpha):
    data = pd.DataFrame()
    pbar = ProgressBar()
    for source in pbar(labels_data.source):
        query = ''' SELECT * FROM newsdata 
                    WHERE source='%s' 
                    LIMIT %i ''' % (source, 
                                    int(labels_data.loc[labels_data.source == source, 'n_articles']*alpha/10))
        data = data.append(execute_query_pandas(path, query))
    return data

In [10]:
alpha = 0.20
path = '../data/NELA2019/data/nela-eng-2019.db'

data = load_articles(path, labels_data, alpha)

100% |########################################################################|


In [14]:
data.to_csv('../data/NELA2019/data/articles_%s.csv' %(str(alpha)[2:]))